In [1]:
%load_ext autoreload
%autoreload 3

import gconcord as cc
import gconcorde as cce
import numpy as np

# import importlib
# importlib.reload(cc)

# true precision matrix
omega = np.zeros((4,4), order="C")
omega[0, 1] = 2.1
omega[1, 2] = -2.1
omega += omega.T
np.fill_diagonal(omega, 4)

# true covariance matrix
sigma = np.linalg.inv(omega)

# observations
np.random.seed(2022)
data = np.random.multivariate_normal([0]*4, sigma, 2000)

# sample covariance matrix
S = np.cov(data, rowvar=False)

In [2]:
lam_mat = 0.1 * np.ones((4, 4), dtype="float64")
np.fill_diagonal(lam_mat, 0)
-np.log(omega.diagonal()).sum() + 0.5*np.matmul(omega, np.matmul(S, omega)).trace() + (lam_mat * np.abs(omega)).sum()

3.27100485543166

In [22]:
%%time
omega_hat, hist = cce.pycce_constant(S, 0.1, epstol=1e-5, maxitr=1000, penalize_diagonal=False)
print(omega_hat)
print(' ')
print('# of iterations:', len(hist))
print(' ')
print('Objective:', hist[-1][2])
print(' ')

[[ 1.91691183  0.76716446  0.          0.        ]
 [ 0.69438322  1.79645523 -0.79857351 -0.        ]
 [ 0.         -0.85002364  1.89956931  0.        ]
 [ 0.         -0.          0.          2.00412908]]
 
# of iterations: 36
 
Objective: -0.4178700483631105
 
CPU times: user 4.55 ms, sys: 17 µs, total: 4.57 ms
Wall time: 4.24 ms


In [11]:
.5*(np.diag(np.diag(omega_hat)) @ omega_hat) + .5*(omega_hat.T @ np.diag(np.diag(omega_hat)))

array([[ 3.67455097,  1.35900749,  0.        ,  0.        ],
       [ 1.35900749,  3.22725139, -1.52464019,  0.        ],
       [ 0.        , -1.52464019,  3.60836355,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  4.01653339]])

In [21]:
%%time
omega_hat, hist = cce.pycceista(S, 0.1, epstol=1e-5, maxitr=1000, penalize_diagonal=False)
print(omega_hat)
print(' ')
print('# of iterations:', len(hist))
print(' ')
print('Objective:', hist[-1][2])
print(' ')

[[ 1.91478508  0.76457886  0.          0.        ]
 [ 0.69005572  1.79184331 -0.79344326 -0.        ]
 [ 0.         -0.84683887  1.89705232  0.        ]
 [ 0.         -0.          0.          2.00411657]]
 
# of iterations: 2001
 
Objective: -0.4178802881739977
 
CPU times: user 4.58 s, sys: 12.1 ms, total: 4.59 s
Wall time: 4.7 s


In [23]:
.5*(np.diag(np.diag(omega_hat)) @ omega_hat) + .5*(omega_hat.T @ np.diag(np.diag(omega_hat)))

array([[ 3.67455097,  1.35900749,  0.        ,  0.        ],
       [ 1.35900749,  3.22725139, -1.52464019,  0.        ],
       [ 0.        , -1.52464019,  3.60836355,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  4.01653339]])